In [1]:
# https://github.com/Azure/aml-real-time-ai/blob/600c67304fdffd92694ab9551fa7b8be9628d47c/notebooks/project-brainwave-transfer-learning.ipynb

In [2]:
import os
import tensorflow as tf
import numpy as np

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Anaconda\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
#!pip install --upgrade azureml-sdk[contrib]
#!pip install --upgrade azureml-sdk[contrib,notebooks,explain,automl,databricks]

In [4]:
#!pip install "tensorflow==1.10" --force
#!pip install --upgrade tensorflow --force

In [5]:
import glob
import imghdr
datadir = os.path.expanduser("~/catsanddogs")

cat_files = glob.glob(os.path.join(datadir, 'PetImages', 'Cat', '*.jpg'))
dog_files = glob.glob(os.path.join(datadir, 'PetImages', 'Dog', '*.jpg'))

# Limit the data set to make the notebook execute quickly.
cat_files = cat_files[:128]
dog_files = dog_files[:128]

# The data set has a few images that are not jpeg. Remove them.
cat_files = [f for f in cat_files if imghdr.what(f) == 'jpeg']
dog_files = [f for f in dog_files if imghdr.what(f) == 'jpeg']

if(not len(cat_files) or not len(dog_files)):
    print("Please download the Kaggle Cats and Dogs dataset form https://www.microsoft.com/en-us/download/details.aspx?id=54765 and extract the zip to " + datadir)    
    raise ValueError("Data not found")
else:
    print(cat_files[0])
    print(dog_files[0])

C:\Users\JBAdmin/catsanddogs\PetImages\Cat\0.jpg
C:\Users\JBAdmin/catsanddogs\PetImages\Dog\0.jpg


In [6]:
# constructing a numpy array as labels
image_paths = cat_files + dog_files
total_files = len(cat_files) + len(dog_files)
labels = np.zeros(total_files)
labels[len(cat_files):] = 1

In [7]:
# We need to preprocess the input file to get it into the form expected by ResNet152. We've provided a default implementation of the preprocessing that you can use.


In [8]:
#!pip install --upgrade azureml-sdk[accel]

In [9]:
# Input images as a two-dimensional tensor containing an arbitrary number of images represented a strings
import azureml.contrib.brainwave.models.utils as utils
#import azureml.accel.models.utils as utils
in_images = tf.placeholder(tf.string)
image_tensors = utils.preprocess_array(in_images)
print(image_tensors.shape)

(?, 224, 224, 3)


In [10]:
from azureml.contrib.brainwave.models import QuantizedResnet152
model_path = os.path.expanduser('~/models')
bwmodel = QuantizedResnet152(model_path, is_frozen = True)
print(bwmodel.version)

1.1.3


In [11]:
features = bwmodel.import_graph_def(input_tensor=image_tensors)

In [12]:
# Pre-Compute features

In [13]:
from tqdm import tqdm

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def read_files(files):
    contents = []
    for path in files:
        with open(path, 'rb') as f:
            contents.append(f.read())
    return contents
        
feature_list = []
with tf.Session() as sess:
    for chunk in tqdm(chunks(image_paths, 5)):
        contents = read_files(chunk)
        result = sess.run([features], feed_dict={in_images: contents})
        feature_list.extend(result[0])

feature_results = np.array(feature_list)
print(feature_results.shape)

51it [07:30,  7.71s/it]


(255, 1, 1, 2048)


In [14]:
# Add and Train the classifier

In [15]:
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten
from keras import optimizers

FC_SIZE = 1024
NUM_CLASSES = 2

model = Sequential()
model.add(Dropout(0.2, input_shape=(1, 1, 2048,)))
model.add(Dense(FC_SIZE, activation='relu', input_dim=(1, 1, 2048,)))
model.add(Flatten())
model.add(Dense(NUM_CLASSES, activation='sigmoid', input_dim=FC_SIZE))

model.compile(optimizer=optimizers.SGD(lr=1e-4,momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

Using TensorFlow backend.


In [16]:
# Prepare the train and test data

In [17]:
from sklearn.model_selection import train_test_split
onehot_labels = np.array([[0,1] if i else [1,0] for i in labels])
X_train, X_test, y_train, y_test = train_test_split(feature_results, onehot_labels, random_state=42, shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(191, 1, 1, 2048) (64, 1, 1, 2048) (191, 2) (64, 2)


In [18]:
# Train the classifier.

In [19]:
model.fit(X_train, y_train, epochs=16, batch_size=32)

Epoch 1/16
191/191 [==============================] - ETA: 27s - loss: 0.7913 - acc: 0.50 - ETA: 2s - loss: 0.7563 - acc: 0.5430 - 6s 30ms/step - loss: 0.7604 - acc: 0.5183
Epoch 2/16
191/191 [==============================] - ETA: 0s - loss: 0.7781 - acc: 0.484 - ETA: 0s - loss: 0.7665 - acc: 0.484 - 0s 764us/step - loss: 0.7618 - acc: 0.4791
Epoch 3/16
191/191 [==============================] - ETA: 0s - loss: 0.7532 - acc: 0.515 - ETA: 0s - loss: 0.7361 - acc: 0.520 - ETA: 0s - loss: 0.7467 - acc: 0.490 - 0s 853us/step - loss: 0.7361 - acc: 0.4948
Epoch 4/16
191/191 [==============================] - ETA: 0s - loss: 0.6708 - acc: 0.656 - ETA: 0s - loss: 0.7141 - acc: 0.510 - 0s 780us/step - loss: 0.7117 - acc: 0.5209
Epoch 5/16
191/191 [==============================] - ETA: 0s - loss: 0.7126 - acc: 0.484 - ETA: 0s - loss: 0.7135 - acc: 0.510 - 0s 783us/step - loss: 0.7101 - acc: 0.5340
Epoch 6/16
191/191 [==============================] - ETA: 0s - loss: 0.7139 - acc: 0.468 - ETA: 

In [20]:
# Test the Classifier

In [21]:
from numpy import argmax

y_probs = model.predict(X_test)
y_prob_max = np.argmax(y_probs, 1)
y_test_max = np.argmax(y_test, 1)
print(y_prob_max)
print(y_test_max)

[0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0
 1 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 1]
[1 0 0 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 0 0
 0 0 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 1 0]


In [22]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import itertools
import matplotlib
from matplotlib import pyplot as plt

# compute a bunch of classification metrics 
def classification_metrics(y_true, y_pred, y_prob):
    cm_dict = {}
    cm_dict['Accuracy'] = accuracy_score(y_true, y_pred)
    cm_dict['Precision'] =  precision_score(y_true, y_pred)
    cm_dict['Recall'] =  recall_score(y_true, y_pred)
    cm_dict['F1'] =  f1_score(y_true, y_pred) 
    cm_dict['AUC'] = roc_auc_score(y_true, y_prob[:,0])
    cm_dict['Confusion Matrix'] = confusion_matrix(y_true, y_pred).tolist()
    return cm_dict

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """Plots a confusion matrix.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    New BSD License - see appendix
    """
    cm_max = cm.max()
    cm_min = cm.min()
    if cm_min > 0: cm_min = 0
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_max = 1
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    thresh = cm_max / 2.
    plt.clim(cm_min, cm_max)

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i,
                 round(cm[i, j], 3),  # round to 3 decimals if they are float
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
cm_dict = classification_metrics(y_test_max, y_prob_max, y_probs)
for m in cm_dict:
    print(m, cm_dict[m])
cm = np.asarray(cm_dict['Confusion Matrix'])
plot_confusion_matrix(cm, ['fail','pass'], normalize=False)

Accuracy 0.46875
Precision 0.5
Recall 0.4117647058823529
F1 0.45161290322580644
AUC 0.5362745098039216
Confusion Matrix [[16, 14], [20, 14]]


<Figure size 640x480 with 2 Axes>

In [23]:
# Service Definition

In [24]:
#!pip install "tensorflow==1.10" --force
#!pip uninstall tensorflow

In [25]:
#!pip install "tensorflow==1.10" --force
#!pip install --upgrade tensorflow --force

In [26]:
from azureml.contrib.brainwave.pipeline import ModelDefinition, TensorflowStage, BrainWaveStage, KerasStage

model_def = ModelDefinition()
model_def.pipeline.append(TensorflowStage(tf.Session(), in_images, image_tensors))
model_def.pipeline.append(BrainWaveStage(tf.Session(), bwmodel))
model_def.pipeline.append(KerasStage(model))

model_def_path = os.path.join(datadir, 'save', 'model_def.zip')
model_def.save(model_def_path)
print(model_def_path)

INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
INFO:tensorflow:Restoring parameters from C:\Users\JBAdmin/models\msfprn152\1.1.3\resnet152_bw
INFO:tensorflow:Froze 4 variables.
Converted 4 variables to const ops.
C:\Users\JBAdmin/catsanddogs\save\model_def.zip


In [27]:
# Deploy

In [28]:
from azureml.core.model import Model
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
model_name = "catsanddogs-model"
service_name = "modelbuild-service"

registered_model = Model.register(ws, model_def_path, model_name)

jbfpgamlws
jbfpga
eastus2
82d7b191-5a8c-4cbf-a9f9-9aa5fb50feaa
Registering model catsanddogs-model


In [29]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.contrib.brainwave import BrainwaveWebservice, BrainwaveImage
try:
    service = Webservice(ws, service_name)
except WebserviceException:
    image_config = BrainwaveImage.image_configuration()
    deployment_config = BrainwaveWebservice.deploy_configuration()
    service = Webservice.deploy_from_model(ws, service_name, [registered_model], image_config, deployment_config)
    service.wait_for_deployment(True)

Creating image
Image creation operation finished for image modelbuild-service:3, operation "Succeeded"
Creating service
Running........................................................................
SucceededFPGA service creation operation finished, operation "Succeeded"


In [30]:
# The service is now running in Azure and ready to serve requests. We can check the address and port.

In [59]:
print(service.ip_address + ':' + str(service.port))

52.170.115.110:80


In [60]:
# Client

In [61]:
from azureml.contrib.brainwave.client import PredictionClient
def run(ip_address, port, input_data):
    client = PredictionClient(service.ipAddress, service.port)
    #client = PredictionClient(ip_address, self.port, False, "")
    #if isinstance(input_data, str):
    #   return client.score_image(input_data)
    #if isinstance(input_data, np.ndarray):
    #   return client.score_numpy_array(input_data)
    #return client.score_file(input_data.read())

In [62]:
#!pip install azureml-contrib-brainwave
#!pip install git+https://github.com/Azure/aml-real-time-ai/tree/master/pythonlib
#!pip install git+https://github.com/Azure/aml-real-time-ai/tree/master/pythonlib

In [63]:
# #import azureml.accel.models.utils as utils
#from azureml.contrib.brainwave.client import PredictionClient
from azureml.contrib.brainwave.client import PredictionClient
#from amlrealtimeai import PredictionClient # for sample classes in GitHub
#from aml-real-time-ai import PredictionClient # for sample classes in GitHub
#from azureml.contrib.brainwave.client import PredictionClient
#print(PredictionClient)
client = PredictionClient(service.ip_address, service.port)

In [64]:
# Request

In [76]:
# Specify an image to classify
print('CATS')
for image_file in cat_files[:8]:
    results = client.score_image(image_file)
    result = 'CORRECT ' if results[0] > results[1] else 'WRONG '
    print(result + str(results))
print('/nDOGS')
for image_file in dog_files[:8]:
    results = client.score_image(image_file)
    result = 'CORRECT ' if results[1] > results[0] else 'WRONG '
    print(result + str(results))

CATS
CORRECT [0.73331517 0.6330011 ]
WRONG [0.3402378 0.513855 ]
CORRECT [0.4637577  0.40981242]
WRONG [0.3930712 0.5067719]
CORRECT [0.576025   0.36346677]
WRONG [0.41783664 0.54699445]
WRONG [0.41781354 0.4930551 ]
CORRECT [0.47046134 0.37351555]
DOGS
WRONG [0.64932466 0.6356455 ]
WRONG [0.70160306 0.5213355 ]
CORRECT [0.46283093 0.5326308 ]
WRONG [0.5426064  0.32584208]
CORRECT [0.39951482 0.6635674 ]
CORRECT [0.4432078  0.44594315]
CORRECT [0.7041569 0.715247 ]
CORRECT [0.3480113  0.73805165]
